In [167]:
import numpy as np
import pandas as pd

In [168]:
df = pd.read_csv("C:/Users/user/Downloads/ds문제풀이/ds문제풀이/ds2(#1)_1/DS1_실습파일/ds1_stat1.csv")
df.head()
df.columns = ["user", "gender", "model_no", "duration", "date"]
df.head()

,user,gender,model_no,duration,date
0,03rhphq1u3,M,SM-S908U,16407,2022-02-21
1,06ubzrmthj,NaN,SM-S908U,70671,2022-02-21
2,08opljrljj,NaN,SM-S908U,30756,2022-02-21
3,08opljrljj,NaN,SM-S908U,53186,2022-02-21
4,08opljrljj,NaN,SM-S908U,59744,2022-02-21


Q1-1 남성과 여성 사용자 수의 합을 구하시오.

In [169]:
df.isna().sum()

user             0
gender      388620
model_no         0
duration         0
date             0
dtype: int64

In [170]:
df[["user","gender"]].dropna().nunique()

user      8985
gender       2
dtype: int64

Q1-2 일주일 간의 실행 횟수가 10회 이상인 사람들의 일주일 간의 총 앱 실행시간의 평균을 구하시오.

In [171]:
con1 = df.groupby("user")["duration"].count()
con2 = df.groupby("user")["duration"].sum()
df_sub = pd.concat([con1, con2], axis = 1)
df_sub.columns = ["count", "duration"]
df_sub.loc[df_sub["count"] >= 10, :]["duration"].mean()

922806.468885915

Q1-3 각 여성 사용자들의 일주일간의 총 앱 실행시간을 계산한 데이터셋을 만들었을 때, 총 실행시간 1990593의 z-통계량(z-score)를 구하시오.

In [172]:
df_sub = df.loc[df["gender"] == "F", :]
female_use = df_sub.groupby("user")["duration"].sum()
(1990593 - female_use.mean()) / female_use.std()

1.3500095592869115

In [173]:
df = pd.read_csv("C:/Users/user/Downloads/ds문제풀이/ds문제풀이/ds2(#1)_1/DS1_실습파일/ds1_stat2.csv")
df.head(2)
df.isna().sum()

inch     0
power    0
dtype: int64

Q2-1 75인치 TV 소비전력의 표본평균을 구하시오.

In [174]:
df.loc[df["inch"] ==75, :]["power"].mean()

358.1416295399329

Q2-2 전체 75인치 TV의 소비전력의 모표준편차는 25kw로 알려져있다. 모평균에 대한 99% 신뢰구간을 (a, b)라 할때 a, b를 각각 구하시오.
(단, P(Z >= 2.58) = 0.005)

In [175]:
sigma = 25
x_bar = df.loc[df["inch"] ==75, :]["power"].mean()
sigma_hat = sigma / df.loc[df["inch"] ==75, :].shape[0] ** 0.5
a = x_bar - sigma_hat * 2.58
b = x_bar + sigma_hat * 2.58
print(a)
print(b)

354.4052448608884
361.87801421897746


화면크기를 독립변수, 소비전력을 종속변수로 하는 선형회귀모델을 학습하고 다음 질문들에 답하시오.


Q3-1 선형회귀모델의 기울기를 구하시오.

In [176]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X = df[["inch"]], y = df[["power"]])
model.coef_

array([[3.1278318]])

Q3-2 SSR, SST, R-squared를 구하시오.

In [177]:
pred = pd.Series(model.predict(df[["inch"]]).reshape(-1))
SST = sum((df["power"] - df["power"].mean())**2)
SSE = sum((df["power"].reset_index(drop=True) - pred)**2)
SSR = SST - SSE
R_squared = SSR/SST
print(SSR)
print(SST)
print(R_squared)

2546695.1787300445
4881757.06925299
0.5216759340135992


Q4-1 'state' 컬럼의 서로 다른 값의 개수는 몇개인지 구하시오.

In [178]:
df = pd.read_csv("C:/Users/user/Downloads/ds문제풀이/ds문제풀이/ds2(#1)_1/DS1_실습파일/ds1_mldata.csv")
df.head(2)

,state,account_length,csc,onlineshopping_beauty,wr_flag,buzz_usage_count,data_spent,app_a_use_count,sms_count,app_b_use_count,call_count,app_c_use_count,call_duration,usage_duration,app_d_use_count,num_service,target
0,TX,66,ATT,0,1,34,36100.00,389.9,21.687314,2001.287587,1104.487108,698.89600,77.44,502.643825,4,1,0
1,VT,89,TMB,0,0,0,28662.49,428.6,11.117396,5392.446695,1420.765545,478.72561,112.36,403.083456,5,2,0


In [179]:
df["state"].nunique()

51

Q4-2 'onlineshopping_beauty'와 'wr_flag'가 모두 1인 고객 중 'target'이 1인 고객의 비율(%)을 구하시오.

In [180]:
con1 = df["onlineshopping_beauty"] == 1
con2 = df["wr_flag"] == 1

df_sub = df.loc[con1 & con2, :]
df_sub.groupby(["onlineshopping_beauty", "wr_flag"])["target"].value_counts(normalize = True).round(4)

onlineshopping_beauty  wr_flag  target
1                      1        1         0.5574
                                0         0.4426
Name: proportion, dtype: float64

In [181]:
df_sub3 = pd.crosstab(df["state"], df["target"], normalize = 0).reset_index().sort_values(1, ascending = False)
df_sub3.iloc[9,:]

target
state          ME
0        0.724138
1        0.275862
Name: 21, dtype: object

Q5-1 

다음 수치형 변수만을 사용하여, 주어진 데이터셋을 train/test 셋으로 분할하여 고객의 이탈여부 'target'을 예측하는 모델을 학습 및 평가하는 미완성 코드이다. 코드를 완성하고 다음 질문에 답하시오.

* 주어진 데이터셋의 행의 순서를 변경하지 말 것.
* 주어진 코드를 변경하지 말 것.

In [188]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score


df = pd.read_csv("C:/Users/user/Downloads/ds문제풀이/ds문제풀이/ds2(#1)_1/DS1_실습파일/ds1_mldata.csv")

df.drop(columns = ["state", "csc"], inplace = True)

X = df.drop(columns = ["target"])
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)
prec = precision_score(y_test, y_pred)
print(f'Precision : {prec: .2f}')



Precision :  0.59


Q5-1 

Train, test 비율을 7:3으로 하여 sklearn.ensemble.RandomForestClassifier 를 학습시키고 test set에서 'target'이 1일 확률이 0.75인 데이터의 개수를 구하시오.


In [189]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv("C:/Users/user/Downloads/ds문제풀이/ds문제풀이/ds2(#1)_1/DS1_실습파일/ds1_mldata.csv")

df.drop(columns = ["state", "csc"], inplace = True)

X = df.drop(columns = ["target"])
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train, y_train)

y_pred = rf.predict_proba(X_test)[:,1]
num = sum(y_pred >= 0.75)
print(f'# of users with probability >= 0.75 is: {num}')

# of users with probability >= 0.75 is: 43
